In [1]:
#### Data Prepare ####

## Libraries
library(tidyverse)
library(dplyr)
library(lubridate)
library(tibble)
library(stringr)
library(stringi)
library(tidytext)
library(ggplot2)
library(superml)
library(reshape2)
library(caret)
library(recommenderlab)
#library(text2vec)

movie0 <- read.csv('movies_metadata.csv')
head(movie0)
m1 <-na.omit(movie0)
summary(m1$vote_count)
summary(m1$vote_average)
m1$release_year <- year(as.Date(m1$release_date))
summary(m1$release_year)
m2 <- m1[complete.cases(m1[,c('id', 'title', 'release_year', 'genres', 'vote_count','vote_average','popularity')]),]
# m2 <- m2[m2$genres != '[]',]
head(m2$genres)
m2$genres1 <- stringi::stri_extract_all(m2$genres, regex = "[A-Z][a-z]+")
#m2$companies1 <- stringi::stri_extract_all(m2$production_companies, regex = "[A-Z][a-z]*[ [A-Z][a-z]*]*")
#m3 <- m2[!is.na(m2$genres1) & !is.na(m2$companies1),]
m3 <- m2[!is.na(m2$genres1),]
genres <- levels(as.factor(unlist(m3$genres1)))
# companies <- levels(as.factor(unlist(m3$companies1)))


genre_df <- tibble::enframe(m3$genres1)%>%
  unnest(value)%>%
  mutate(temp=1)%>%
  pivot_wider(names_from=value, values_from=temp, values_fill=list(temp=0))

m4 <- cbind(m3,genre_df)

movie1 <- cbind(m3, genre_df)


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Loading required package: R6


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: arules


Attaching package: ‘arules’


The following 

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,⋯,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>
1,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",⋯,1995-10-30,373554033,81,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Toy Story,False,7.7,5415
2,False,,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",⋯,1995-12-15,262797249,104,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
3,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",,15602,tt0113228,en,Grumpier Old Men,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.",⋯,1995-12-22,0,101,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92
4,False,,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]",,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.",⋯,1995-12-22,81452156,127,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and never let you forget it.,Waiting to Exhale,False,6.1,34
5,False,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}",0,"[{'id': 35, 'name': 'Comedy'}]",,11862,tt0113041,en,Father of the Bride Part II,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home, but that's a plan that -- like George -- will have to change with the arrival of both a grandchild and a kid of his own.",⋯,1995-02-10,76578911,106,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's In For The Surprise Of His Life!,Father of the Bride Part II,False,5.7,173
6,False,,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    0.0     3.0    10.0   110.5    34.0 14075.0 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  0.000   5.000   6.000   5.629   6.800  10.000 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
   1874    1978    2001    1992    2010    2020      73 

[1] "[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"                        
[2] "[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]"                       
[3] "[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]"                                                        
[4] "[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]"                           
[5] "[{'id': 35, 'name': 'Comedy'}]"                                                                                          
[6] "[{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 18, 'name': 'Drama'}, {'id': 53, 'name': 'Thriller'}]"

In [2]:
#### Simple Recommender ####

simple_recommender <- function(data=m4, genre, rec_num, by_vote_avg=F, by_popularity=T, year_after=NULL, ...){
  
  if (is.null(year_after)) {generic_df <- data[data[,genre]==1 & data$vote_count>=50, ]}
  else {generic_df <- data[data[,genre]==1 & data$vote_count>=50 & data$release_year >= year_after, ]}
  
  
  if (by_vote_avg==T) {generic_df <-  generic_df[order(generic_df$vote_average,as.numeric(generic_df$popularity) , decreasing = T),]}
  else  {generic_df <-  generic_df[order(as.numeric(generic_df$popularity, generic_df$vote_average), decreasing = T),]}
  rec <- generic_df[1:rec_num, c('title', 'release_year', 'genres1','vote_average', 'popularity')]
  row.names(rec) <- NULL
  return(rec)
}

simple_recommender(genre = 'Action', rec_num = 10, year_after = 2015)



title,release_year,genres1,vote_average,popularity
<chr>,<dbl>,<list>,<dbl>,<chr>
Wonder Woman,2017,"Action , Adventure, Fantasy",7.2,294.337037
Baby Driver,2017,"Action, Crime",7.2,228.032744
Deadpool,2016,"Action , Adventure, Comedy",7.4,187.860492
Guardians of the Galaxy Vol. 2,2017,"Action , Adventure, Comedy , Science , Fiction",7.6,185.330992
Captain America: Civil War,2016,"Adventure, Action , Science , Fiction",7.1,145.882135
Pirates of the Caribbean: Dead Men Tell No Tales,2017,"Adventure, Action , Fantasy , Comedy",6.6,133.82782
Captain Underpants: The First Epic Movie,2017,"Action , Animation, Comedy , Family",6.5,88.561239
Ghost in the Shell,2017,"Action , Science , Fiction , Thriller",5.9,68.726676
Boyka: Undisputed IV,2016,"Thriller, Action , Crime",5.8,67.955052


In [3]:
#### Content-Based ####
m_temp <- m3[,c('id', 'title', 'release_year','genres1', 'overview', 'tagline', 'vote_count', 'vote_average', 'popularity')]
link <- read.csv('links_small.csv')
movie1 <- merge(m_temp, link, by.x = c('id'), by.y=c('tmdbId'))
movie1$description <- paste(movie1$overview, movie1$tagline)
saveRDS(movie1, 'movie1.Rds')
m_temp <- movie1[,c('title','description')]

temp <- m_temp%>%tidytext::unnest_tokens(word, description)%>%anti_join(stop_words)
freq_temp <- temp%>%count(word, sort = T)
wordcloud2::wordcloud2(freq_temp)


tfv <- superml::TfIdfVectorizer$new(max_features = 1000, remove_stopwords = T)
tf_mat <- tfv$fit_transform(m_temp$description)

cos_sim <- function(m1,m2, ...){
  mat <- tcrossprod(m1,m2)
  t1 <- sqrt(apply(m1,1,crossprod))
  t2 <- sqrt(apply(m2, 1, crossprod))
  mat/outer(t1,t2)
}

cos_sim_mat <- cos_sim(tf_mat, tf_mat)

titles <- as.factor(movie1$title)
sim_df <- as.data.frame(cos_sim_mat)

content_recommender <- function(title, rec_num=10, description=F, ...){
  
  if (description==F) {description <- NULL}
  else description='descripton'
  
  sim_scores <- as.vector(t(sim_df[which(titles==title),]))
  sim_by_idx <- order(sim_scores,decreasing = T)
  rec_idx <- sim_by_idx[2:(rec_num+1)]
  rec_movies <- movie1[rec_idx, c('title', 'release_year', 'genres1',description)]
  row.names(rec_movies) <- NULL
  return(rec_movies)
}

content_recommender('Toy Story',5)



Joining, by = "word"



HTML widgets cannot be represented in plain text (need html)

title,release_year,genres1
<chr>,<dbl>,<list>
The Proposal,2009,"Comedy , Romance, Drama"
Singles,1992,"Comedy , Drama , Romance"
Four Rooms,1995,"Crime , Comedy"
"Paris, Texas",1984,Drama
Dirty Dancing: Havana Nights,2004,"Drama , Romance"


In [4]:

#### Collaborative-Filtering (User-Based CF) ####
rating <- read.csv('ratings_small.csv')
id_rating_temp <- merge(rating, link, by='movieId')
movie_rating <- merge(id_rating_temp, movie1, by='movieId')

cf_df <- reshape2::dcast(rating, userId~movieId, value.var='rating')
cf_mat <- as.matrix(cf_df[,-1])
# dim(ucf_mat)
cf_mat <- as(cf_mat, "realRatingMatrix")
# cf_mat@data@Dimnames[[1]] <- unique(rating$userId) mute the userIds from the rating matrix if needed

userIds <- unique(rating$userId)
ubcf_model <- recommenderlab::Recommender(cf_mat, method='UBCF', param='cosine')
UBCF_Recommender <- function(userId, rec_num=10, description=F, ...){
  
  if (description==F) {description <- NULL}
  else description='description'
  
  rec <- predict(ubcf_model, cf_mat[userId], n=rec_num)
  rec_list <- as(rec, 'list')
  rec_movieId <- rec_list[[1]]
  rec_movies <- movie_rating[movie_rating$movieId %in% rec_movieId, c('title','release_year', 'genres1', description)]
  rec_movies <- unique(rec_movies)
  # rec_movies <- rec_movies[,c('title','release_year', 'genres1', description)]
  row.names(rec_movies) <- NULL
  # rec_movies <- as.data.frame(rec_movies)
  return(rec_movies)
}

UBCF_Recommender(11,5,T)



title,release_year,genres1,description
<chr>,<dbl>,<list>,<chr>
Poltergeist,1982,Horror,"Steve Freeling lives with his wife, Diane, and their three children, Dana, Robbie, and Carol Anne, in Southern California where he sells houses for the company that built the neighborhood. It starts with just a few odd occurrences, such as broken dishes and furniture moving around by itself. However, when he realizes that something truly evil haunts his home, Steve calls in a team of parapsychologists led by Dr. Lesh to help before it's too late. They're here."
American Movie,1999,Documentary,"AMERICAN MOVIE is the story of filmmaker Mark Borchardt, his mission, and his dream. Spanning over two years of intense struggle with his film, his family, financial decline, and spiritual crisis, AMERICAN MOVIE is a portrayal of ambition, obsession, excess, and one man's quest for the American Dream."
Best in Show,2000,Comedy,"The tension is palpable, the excitement is mounting and the heady scent of competition is in the air as hundreds of eager contestants from across America prepare to take part in what is undoubtedly one of the greatest events of their lives -- the Mayflower Dog Show. The canine contestants and their owners are as wondrously diverse as the great country that has bred them. Some pets deserve a little more respect than others."
Bully,2001,"Crime, Drama","After finding himself at the constant abuse of his best friend Bobby, Marty has become fed up with his friend's twisted ways. His girlfriend, a victim of Bobby's often cruel ways, couldn't agree more and they strategize murdering Bobby. It's 4 a.m... do you know where your kids are?"
The Dark Knight,2008,"Drama , Action , Crime , Thriller","Batman raises the stakes in his war on crime. With the help of Lt. Jim Gordon and District Attorney Harvey Dent, Batman sets out to dismantle the remaining criminal organizations that plague the streets. The partnership proves to be effective, but they soon find themselves prey to a reign of chaos unleashed by a rising criminal mastermind known to the terrified citizens of Gotham as the Joker. Why So Serious?"


In [5]:

#### Hybrid Recommender (ContentBased + CollaborativeFiltering)####

hybrid_recommender <- function(userId, title, rec_num, description=F, ...){
  
  if (description==F) {description <- NULL}
  else description='description'
  
  rec_step1 <- UBCF_Recommender(userId, 50, T)
  rec_step1.2 <- rbind(movie1[which(movie1$title==title), 
                              c('title', 'release_year', 'genres1', 'description')], rec_step1)
  
  
  tf_mat_temp <- tfv$fit_transform(rec_step1.2$description)
  cos_sim_mat_temp <- cos_sim(tf_mat_temp, tf_mat_temp)
  sim_df_temp <- as.data.frame(cos_sim_mat_temp)
  
  
  sim_scores <- as.vector(t(sim_df_temp[1,]))
  sim_by_idx <- order(sim_scores, decreasing = T)
  rec_idx <- sim_by_idx[2:(rec_num+1)]
  rec_step2 <- movie1[rec_idx, c('title', 'release_year', 'genres1', description)]
  row.names(rec_step2) <- NULL
  return(rec_step2)
  
}
hybrid_recommender(55, titles[100], 5, T)




title,release_year,genres1,description
<chr>,<dbl>,<list>,<chr>
Hot Rod,2007,"Action, Comedy","For Rod Kimball, performing stunts is a way of life, even though he is rather accident-prone. Poor Rod cannot even get any respect from his stepfather, Frank, who beats him up in weekly sparring matches. When Frank falls ill, Rod devises his most outrageous stunt yet to raise money for Frank's operation -- and then Rod will kick Frank's butt. Smack destiny in the face."
My Sister's Keeper,2009,Drama,"Sara and Brian live an idyllic life with their young son and daughter. But their family is rocked by sudden, heartbreaking news that forces them to make a difficult and unorthodox choice in order to save their baby girl's life. The parents' desperate decision raises both ethical and moral questions and rips away at the foundation of their relationship. Their actions ultimately set off a court case that threatens to tear the family apart, while revealing surprising truths that challenge everyone's perceptions of love and loyalty and give new meaning to the definition of healing. A tale of life, love, and letting go."
Date Movie,2006,Comedy,"Spoof of romantic comedies which focuses on a man (Campbell), his crush (Hannigan), his parents (Coolidge, Willard), and her father (Griffin). An Unprotected Comedy"
The Pacifier,2005,"Action , Comedy , Drama , Family , Thriller",Disgraced Navy SEAL Shane Wolfe is handed a new assignment: Protect the five Plummer kids from enemies of their recently deceased father -- a government scientist whose top-secret experiment remains hidden in the kids' house. Welcome to the infantry.
The Messenger: The Story of Joan of Arc,1999,"Adventure, Drama , Action , History , War",In 1429 a teenage girl from a remote French village stood before her King with a message she claimed came from God; that she would defeat the world's greatest army and liberate her country from its political and religious turmoil. Following her mission to reclaim god's dimished kingdom - through her amazing victories until her violent and untimely death.
